<!-- <a href="https://media.suckhoecong.vn/Images/Uploaded/Share/2016/05/11/michelin_star.png">Image</a> -->

# Instruction:
### Overview of Michelin: (and the data)

### Objectives:
- What is the most Michelin rank existed?
- What is the highest/lowest price?
- Which country does it have most/lest Michelin restaurants?
- What is the most beloved cuisine style? (based on Michelin rank)
### Notes:

# First Setup

### Setup Library - Config - Import data

In [2]:
# Setup Library
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import re

from count_sum import *
from info_enhance import *
from API_Get import *

import folium

SyntaxError: invalid syntax (3123363358.py, line 1)

In [ ]:
# Config
pd.set_option('display.float_format', lambda x: '%.2f' % x)
sns.set(style="darkgrid")

In [ ]:
# Import Data
df = pd.read_csv('michelin_my_maps.csv')

### Data first look

In [ ]:
print('Data Overview:-----------------------------------------')
display(df.head(5))
print('Data Structure:-----------------------------------------')
display(infoOut(df,vunique=True))

### Null-data Solution & Reconfig:

In [ ]:
# Null-data explore:
print('Percentage of null: ----------------------------------')
null_per = df.isnull().sum()*100/len(df)
null_per = null_per.to_frame()
null_per.columns = ['Percent']
null_per.index.names = ['Name']
null_per.reset_index(inplace=True)
display(null_per)

In [ ]:
# Drop Null-data and Reconfig format
df = df.dropna(subset=['MinPrice','MaxPrice','Currency']).reset_index().drop(columns='index')
df['MinPrice'] = df['MinPrice'].str.replace(',','')
df['MinPrice'] = df['MinPrice'].astype(str).astype('float64')
df['MaxPrice'] = df['MaxPrice'].str.replace(',','')
df['MaxPrice'] = df['MaxPrice'].astype(str).astype('float64')

### Null solved data

In [ ]:
print('Overview: ===============================')
display(df.head(3))
print('Summary: ===============================')
df_summary = infoOut(df,vunique=True)
display(df_summary)

### Get currency to exchange


In [ ]:
# Dict to exchange
# Curr_ex_dict = dict()
# for index,value in enumerate(lst_plus):
#     dic_value = get_connection(CE = value)
#     Curr_ex_dict[value] = dic_value[value]
# display(Curr_ex_dict)

# Tempt exchange dict
Curr_ex_dict = {
       "EUR_USD": 1.0576,
       "GBP_USD": 1.2462,
       "MOP_USD": 0.1237,
       "HKD_USD": 0.1274,
       "JPY_USD": 0.0078,
       "KRW_USD": 0.0008,
       "CNY_USD": 0.1489,
       "USD_USD": 1,
       "SEK_USD": 0.1007,
       "NOK_USD": 0.1031,
       "DKK_USD": 0.1421,
       "SGD_USD": 0.7244,
       "TWD_USD": 0.0337,
       "CHF_USD": 1.0283,
       "THB_USD": 0.0290,
       "BRL_USD": 0.2031,
       "CZK_USD": 0.0428,
       "ISK_USD": 0.0076,
       "HRK_USD": 0.1403,
       "HUF_USD": 0.0028,
       "PLN_USD": 0.2275,
       "RSD_USD": 0.0090
}

# Process

### Create new data

In [ ]:
# First data
db = df.iloc[:,:]
da = db['Address']

# New data-field: Nation
db['Nation'] = [re.findall(r", \w(?:[A-Za-z])+", da)[-1].replace(', ','') for da in da]

# New data-field: CtUSD (Currency to USD)
db['CtUSD'] = db['Currency']+'_USD'

# New data-field: MinPrice_USD & MaxPrice_USD
db['MinPrice_USD'] = [db['MinPrice'][i]*Curr_ex_dict.get(db['CtUSD'][i]) for i in range(len(db))]
db['MaxPrice_USD'] = [db['MaxPrice'][i]*Curr_ex_dict.get(db['CtUSD'][i]) for i in range(len(db))]

# Split Cuisine-field
db['Cuisine'] = [db['Cuisine'][i].split(', ') for i in range(len(db))]

# Filter&Sort data
cols = ['Name','Location', 'Nation', 'MinPrice_USD', 'MaxPrice_USD','Cuisine', 'Longitude', 'Latitude', 'Award']
db = db.loc[:,cols]

### Summary based on Nation

In [ ]:
# Based on Nations (no ranking)
by_Nation = db\
            .groupby('Nation')\
            .agg({"Name":"count","MinPrice_USD":"min","MaxPrice_USD":"max","Longitude":"mean","Latitude":"mean"})\
            .reset_index()
by_Nation.columns = ['Nations','Count','MinP','MaxP','Long','Lat']

# Based on Nations (with ranking)
by_Nation_rank_3 = db[db['Award'] == '3 MICHELIN Stars']\
            .groupby('Nation')\
            .agg({"Name":"count","MinPrice_USD":"min","MaxPrice_USD":"max","Longitude":"mean","Latitude":"mean"})\
            .reset_index()
by_Nation_rank_3.columns = ['Nations','Count','MinP','MaxP','Long','Lat']

by_Nation_rank_2 = db[db['Award'] == '2 MICHELIN Stars']\
            .groupby('Nation')\
            .agg({"Name":"count","MinPrice_USD":"min","MaxPrice_USD":"max","Longitude":"mean","Latitude":"mean"})\
            .reset_index()
by_Nation_rank_2.columns = ['Nations','Count','MinP','MaxP','Long','Lat']

by_Nation_rank_1 = db[db['Award'] == '1 MICHELIN Star']\
            .groupby('Nation')\
            .agg({"Name":"count","MinPrice_USD":"min","MaxPrice_USD":"max","Longitude":"mean","Latitude":"mean"})\
            .reset_index()
by_Nation_rank_1.columns = ['Nations','Count','MinP','MaxP','Long','Lat']

by_Nation_rank_0 = db[db['Award'] == 'Bib Gourmand']\
            .groupby('Nation')\
            .agg({"Name":"count","MinPrice_USD":"min","MaxPrice_USD":"max","Longitude":"mean","Latitude":"mean"})\
            .reset_index()
by_Nation_rank_0.columns = ['Nations','Count','MinP','MaxP','Long','Lat']

### Summary based on Award

In [ ]:
cols = ['Award','MinPrice_USD','MaxPrice_USD']
Award_db = db[cols]
Award_db = pd.melt(Award_db, id_vars =['Name'], value_vars =['Course'])

# Conclusion:

### How Michelin stars (without ranking) distributes in nations

In [ ]:
# Make an empty map
m_Nations = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)

# add marker one by one on the map
data= by_Nation.iloc[:,:]

for i in range(0,len(data)):
   folium.Circle(
      location=[data.iloc[i]['Lat'], data.iloc[i]['Long']],
      popup=data.iloc[i]['Name'],
      radius=float(data.iloc[i]['Count'])*200,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(m_Nations)

# Show the map again
m_Nations

### How Michelin stars (with ranking) distributes in nations

In [ ]:
# Make an empty map
m_Nations = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)

# add marker one by one on the map
data_3= by_Nation_rank_3.iloc[:,:]
data_2= by_Nation_rank_2.iloc[:,:]
data_1= by_Nation_rank_1.iloc[:,:]
data_0= by_Nation_rank_0.iloc[:,:]

for i in range(0,len(data_3)):
   data == data_3.iloc[:,:]
   folium.Circle(
      location=[data.iloc[i]['Lat'], data.iloc[i]['Long']],
      popup=data.iloc[i]['Name'],
      radius=float(data.iloc[i]['Count'])*200,
      color='crimson',
      fill=True,
      fill_color='crimson'
   ).add_to(m_Nations)

for i in range(0,len(data_2)):
   data == data_1.iloc[:,:]
   folium.Circle(
      location=[data.iloc[i]['Lat'], data.iloc[i]['Long']],
      popup=data.iloc[i]['Name'],
      radius=float(data.iloc[i]['Count'])*200,
      color='azure',
      fill=True,
      fill_color='azure'
   ).add_to(m_Nations)

for i in range(0,len(data_1)):
   data == data_1.iloc[:,:]
   folium.Circle(
      location=[data.iloc[i]['Lat'], data.iloc[i]['Long']],
      popup=data.iloc[i]['Name'],
      radius=float(data.iloc[i]['Count'])*200,
      color='burlywood',
      fill=True,
      fill_color='burlywood'
   ).add_to(m_Nations)

for i in range(0,len(data_0)):
   data == data_0.iloc[:,:]
   folium.Circle(
      location=[data.iloc[i]['Lat'], data.iloc[i]['Long']],
      popup=data.iloc[i]['Name'],
      radius=float(data.iloc[i]['Count'])*200,
      color='gray',
      fill=True,
      fill_color='gray'
   ).add_to(m_Nations)

# Show the map again
m_Nations

### What is the most Michelin rank existed?

In [ ]:
sns.violinplot(x = )